In [1]:
# Imports and config
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 120)
sns.set_theme(style="whitegrid", rc={"figure.figsize": (10,5)})


In [2]:
#  Load CSV from local path (download from Drive first)
SRC = "flights_sample_3m.csv" 
df = pd.read_csv(SRC)
rows, cols = df.shape
print(f"Loaded: {rows:,} rows, {cols} columns")
df.head(3)


Loaded: 3,000,000 rows, 32 columns


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",1155,1151.0,-4.0,19.0,1210.0,1443.0,4.0,1501,1447.0,-14.0,0.0,NaN,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",2120,2114.0,-6.0,9.0,2123.0,2232.0,38.0,2315,2310.0,-5.0,0.0,NaN,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",954,1000.0,6.0,20.0,1020.0,1247.0,5.0,1252,1252.0,0.0,0.0,NaN,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN


In [3]:
# Schema, dtypes, nulls, memory
schema = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes.astype(str),
    "nulls": df.isna().sum().values,
    "null_pct": (df.isna().mean()*100).round(2).values
})
schema.sort_values("nulls", ascending=False).head(20)

,column,dtype,nulls,null_pct
CANCELLATION_CODE,CANCELLATION_CODE,object,2920860,97.36
DELAY_DUE_LATE_AIRCRAFT,DELAY_DUE_LATE_AIRCRAFT,float64,2466137,82.20
DELAY_DUE_SECURITY,DELAY_DUE_SECURITY,float64,2466137,82.20
DELAY_DUE_NAS,DELAY_DUE_NAS,float64,2466137,82.20
DELAY_DUE_WEATHER,DELAY_DUE_WEATHER,float64,2466137,82.20
DELAY_DUE_CARRIER,DELAY_DUE_CARRIER,float64,2466137,82.20
AIR_TIME,AIR_TIME,float64,86198,2.87
ELAPSED_TIME,ELAPSED_TIME,float64,86198,2.87
ARR_DELAY,ARR_DELAY,float64,86198,2.87
WHEELS_ON,WHEELS_ON,float64,79944,2.66


In [4]:
# Quick sampling and memory-friendly casts
df = df.copy()

# Trim strings
for c in df.select_dtypes(include=["object"]).columns:
    df[c] = df[c].astype(str).str.strip()

# Numeric coercions where present
num_candidates = ["DEP_DELAY","ARR_DELAY","TAXI_OUT","TAXI_IN","CRS_ELAPSED_TIME","ELAPSED_TIME","AIR_TIME","DISTANCE",
                  "DELAY_DUE_CARRIER","DELAY_DUE_WEATHER","DELAY_DUE_NAS","DELAY_DUE_SECURITY","DELAY_DUE_LATE_AIRCRAFT"]
for c in [x for x in num_candidates if x in df.columns]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Dates
if "FL_DATE" in df.columns:
    df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], errors="coerce")

df.sample(5, random_state=42)


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
2945667,2021-05-04,JetBlue Airways,JetBlue Airways: B6,B6,20409,384,MCO,"Orlando, FL",JFK,"New York, NY",1551,1558.0,7.0,15.0,1613.0,1822.0,8.0,1830,1830.0,0.0,0.0,nan,0.0,159.0,152.0,129.0,944.0,NaN,NaN,NaN,NaN,NaN
2352586,2019-11-26,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,705,FLL,"Fort Lauderdale, FL",DTW,"Detroit, MI",800,755.0,-5.0,15.0,810.0,1040.0,4.0,1100,1044.0,-16.0,0.0,nan,0.0,180.0,169.0,150.0,1127.0,NaN,NaN,NaN,NaN,NaN
1531260,2023-06-18,Southwest Airlines Co.,Southwest Airlines Co.: WN,WN,19393,1926,SMF,"Sacramento, CA",LAS,"Las Vegas, NV",2155,2151.0,-4.0,10.0,2201.0,2300.0,4.0,2315,2304.0,-11.0,0.0,nan,0.0,80.0,73.0,59.0,397.0,NaN,NaN,NaN,NaN,NaN
941910,2019-07-28,SkyWest Airlines Inc.,SkyWest Airlines Inc.: OO,OO,20304,4459,OKC,"Oklahoma City, OK",DTW,"Detroit, MI",550,546.0,-4.0,11.0,557.0,859.0,8.0,924,907.0,-17.0,0.0,nan,0.0,154.0,141.0,122.0,900.0,NaN,NaN,NaN,NaN,NaN
2582125,2023-03-17,JetBlue Airways,JetBlue Airways: B6,B6,20409,277,FLL,"Fort Lauderdale, FL",SFO,"San Francisco, CA",2049,2049.0,0.0,21.0,2110.0,24.0,7.0,13,31.0,18.0,0.0,nan,0.0,384.0,402.0,374.0,2584.0,0.0,0.0,18.0,0.0,0.0


In [5]:
# Handle nulls in core fields (delays, cancel, diverted)
# Fill binary flags, keep minutes as NaN (to preserve "unknown" vs 0)
for c in ["CANCELLED","DIVERTED"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

# Ensure delay minutes are numeric; NaNs kept
for c in [x for x in ["DEP_DELAY","ARR_DELAY","TAXI_OUT","TAXI_IN"] if x in df.columns]:
    df[c] = pd.to_numeric(df[c], errors="coerce")


In [6]:
import numpy as np

# If DEP_TIME exists in HHMM numeric form, create DEP_TIME_STR
def to_hhmm(v):
    try:
        v = int(v)
        hh = v // 100
        mm = v % 100
        if 0 <= hh <= 23 and 0 <= mm <= 59:
            return f"{hh:02d}:{mm:02d}"
        return np.nan
    except Exception:
        return np.nan

if "DEP_TIME" in df.columns and "DEP_TIME_STR" not in df.columns:
    df["DEP_TIME_STR"] = df["DEP_TIME"].apply(to_hhmm)

# Hour from DEP_TIME_STR
def hour_from_str(s):
    try:
        return int(str(s).split(":")[0])
    except Exception:
        return np.nan

if "DEP_TIME_STR" in df.columns and "DEP_HOUR" not in df.columns:
    df["DEP_HOUR"] = df["DEP_TIME_STR"].apply(hour_from_str)

# Month/DayOfWeek from FL_DATE if present
if "FL_DATE" in df.columns:
    df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], errors="coerce")
    df["Month"] = df["FL_DATE"].dt.month
    df["DayOfWeek"] = df["FL_DATE"].dt.dayofweek + 1  # 1-7

# Route helper
if "ORIGIN" in df.columns and "DEST" in df.columns:
    df["ROUTE"] = df["ORIGIN"].astype(str).str.strip() + "-" + df["DEST"].astype(str).str.strip()

# Quick preview (safe selection)
cols = [c for c in ["FL_DATE","Month","DayOfWeek","DEP_TIME_STR","DEP_HOUR","ROUTE"] if c in df.columns]
df[cols].head(3)


,FL_DATE,Month,DayOfWeek,DEP_TIME_STR,DEP_HOUR,ROUTE
0,2019-01-09,1,3,11:51,11.0,FLL-EWR
1,2022-11-19,11,6,21:14,21.0,MSP-SEA
2,2022-07-22,7,5,10:00,10.0,DEN-MSP


In [7]:
# Normalize DELAY_REASON to exactly one value per row
minute_map = {
    "DELAY_DUE_CARRIER": "Carrier",
    "DELAY_DUE_WEATHER": "Weather",
    "DELAY_DUE_NAS": "NAS",
    "DELAY_DUE_SECURITY": "Security",
    "DELAY_DUE_LATE_AIRCRAFT": "Late Aircraft",
}
minute_cols = [c for c in minute_map if c in df.columns]

def reasons_from_minutes(row):
    out = []
    for c in minute_cols:
        v = row[c]
        if pd.notna(v) and float(v) > 0:
            out.append(minute_map[c])
    return out

made = False
if minute_cols:
    df["__REASONS"] = df.apply(reasons_from_minutes, axis=1)
    if df["__REASONS"].map(len).sum() > 0:
        df = df.explode("__REASONS", ignore_index=True)
        df["DELAY_REASON"] = df["__REASONS"].fillna("").astype(str).str.strip().str.title()
        df = df.drop(columns=["__REASONS"])
        made = True

# If not made, split existing DELAY_REASON text (comma/semicolon/pipe)
import re
if not made and "DELAY_REASON" in df.columns:
    def to_list(s):
        if pd.isna(s): return []
        s = str(s).strip()
        if s == "": return []
        return [p.strip() for p in re.split(r"[;,|]+", s) if p.strip()]
    df["__REASONS"] = df["DELAY_REASON"].apply(to_list)
    if df["__REASONS"].map(len).sum() > 0:
        df = df.explode("__REASONS", ignore_index=True)
        df["DELAY_REASON"] = df["__REASONS"].fillna("").astype(str).str.strip().str.title()
        df = df.drop(columns=["__REASONS"])
        made = True

# Fallback: deterministic classification if still not made
if not made:
    def fallback(row):
        if "CANCELLED" in df.columns and row.get("CANCELLED", 0) == 1: return "Cancelled"
        if "DIVERTED" in df.columns and row.get("DIVERTED", 0) == 1: return "Diverted"
        d, a = row.get("DEP_DELAY", np.nan), row.get("ARR_DELAY", np.nan)
        if pd.notna(d) and d > 0 and pd.notna(a) and a > 0: return "Departure & Arrival"
        if pd.notna(d) and d > 0: return "Departure Delay"
        if pd.notna(a) and a > 0: return "Arrival Delay"
        return "On Time"
    df["DELAY_REASON"] = df.apply(fallback, axis=1)

# Guarantees scalar strings
df["DELAY_REASON"] = df["DELAY_REASON"].astype(str).str.strip()
assert not df["DELAY_REASON"].apply(lambda v: isinstance(v, list)).any()

df["DELAY_REASON"].value_counts().head(10)


DELAY_REASON
                 2466137
Carrier           296951
Late Aircraft     259014
Nas               256477
Weather            31428
Security            2759
Name: count, dtype: int64

In [8]:
# Optional: cap extreme delays to stabilize visuals
for c in ["ARR_DELAY","DEP_DELAY"]:
    if c in df.columns:
        q1, q99 = df[c].quantile(0.01), df[c].quantile(0.99)
        df[c] = df[c].clip(lower=q1, upper=q99)

# Ensure standard numeric columns are numeric
for c in ["ARR_DELAY","DEP_DELAY","TAXI_OUT","TAXI_IN","ELAPSED_TIME","CRS_ELAPSED_TIME","AIR_TIME","DISTANCE"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

print("Ready for visuals.")


Ready for visuals.


In [11]:
# Deliverables: cleaned CSV and feature dictionary
OUT = "flight_delay_3m_clean_single_reason.csv"
df.to_csv(OUT, index=False)
print("Saved:", OUT)

feat = pd.DataFrame({
    "feature": ["FL_DATE","AIRLINE","AIRLINE_CODE","DEP_DELAY","ARR_DELAY","DELAY_REASON","Month","DayOfWeek","DEP_HOUR","ROUTE"],
    "type":    ["date","string","string","number","number","category","number","number","number","string"],
    "notes":   [
        "Flight date","Airline name","Airline code","Departure delay (min)","Arrival delay (min)",
        "Single cause label per row (derived/split/fallback)","Month 1–12","1=Mon..7=Sun","Hour of departure","ORIGIN-DEST"
    ]
})
feat.to_csv("feature_dictionary.csv", index=False)
with open("summary_preprocessing.md","w",encoding="utf-8") as f:
    f.write("# Summary\n- Cleaned dataset with single-value DELAY_REASON\n- Engineered Month, DayOfWeek, DEP_HOUR, ROUTE\n- Saved feature_dictionary.csv and this summary\n")
print("Saved: feature_dictionary.csv, summary_preprocessing.md")


Saved: flight_delay_3m_clean_single_reason.csv
Saved: feature_dictionary.csv, summary_preprocessing.md
